In [1]:
import psycopg2
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.offline as pyo
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots


In [2]:
conn = psycopg2.connect(
           dbname="qdap_test",
           user="amt",
           password="your_password",
           host="192.168.2.23",
           port="5432"
       )

# Create a cursor object using the cursor() method
cursor = conn.cursor()

def fetch_options(cursor, symbol, expiry):
    query = '''
        SELECT * 
        FROM ohlcv_options_per_minute oopm
        WHERE symbol = %s
        AND expiry_type = 'I'
        AND expiry = %s
        ORDER BY date_timestamp;
        '''
    cursor.execute(query,(symbol,expiry))
    rows = cursor.fetchall()
    df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
    return df
# Create a cursor object
cursor = conn.cursor()

def fetch_futures(cursor, symbol, x):
    query = '''
        SELECT *
        FROM ohlcv_future_per_minute ofpm 
        WHERE ofpm.symbol = %s
        AND ofpm.expiry_type = 'I'
        AND DATE(ofpm.expiry) = %s
        ORDER BY date_timestamp ASC
    '''
    # Execute the query with parameters as a tuple
    cursor.execute(query, (symbol, x))
    rows = cursor.fetchall()
    df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
    return df

# Define the fetch_expiries function
def fetch_expiries(cursor, symbol):
    query = f'''
        SELECT DISTINCT ofpem.expiry 
        FROM ohlcv_future_per_minute ofpem 
        WHERE ofpem.symbol = '{symbol}'
        AND ofpem.expiry_type = 'I'
        GROUP BY ofpem.expiry 
    '''
    cursor.execute(query)
    rows = cursor.fetchall()
    df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
    return df

In [4]:
symbol = 'BANKNIFTY'  # Replace with your symbol
expiry_future = fetch_expiries(cursor, symbol)

# Convert the 'expiry' column to datetime and then to date
expiry_future['expiry'] = pd.to_datetime(expiry_future['expiry']).dt.date

# Print the expiry dates with their indices
for i, row in expiry_future.iterrows():
    print(i, row.iloc[0], sep=' = ', end=' | ')
    if (i + 1) % 5 == 0:
        print()

0 = 2022-05-26 | 1 = 2022-06-30 | 2 = 2022-07-28 | 3 = 2022-08-25 | 4 = 2022-09-29 | 
5 = 2022-10-27 | 6 = 2022-11-24 | 7 = 2022-12-29 | 8 = 2023-01-25 | 9 = 2023-02-23 | 
10 = 2023-03-29 | 11 = 2023-04-27 | 12 = 2023-05-25 | 13 = 2023-06-28 | 14 = 2023-06-29 | 
15 = 2023-07-27 | 16 = 2023-08-31 | 17 = 2023-09-28 | 18 = 2023-10-26 | 19 = 2023-11-30 | 
20 = 2023-12-28 | 21 = 2024-01-25 | 22 = 2024-02-29 | 23 = 2024-03-27 | 24 = 2024-04-24 | 
25 = 2024-05-29 | 26 = 2024-06-26 | 27 = 2024-07-31 | 

In [5]:
index = 12
try:
    index
except NameError:
    index = 12  # Default value if not provided

expiry_date = expiry_future.iloc[index]['expiry']
print(f"Selected expiry date: {expiry_date} of type {type(expiry_date)}")
print(f"Selected expiry date: {expiry_date} of type {type(expiry_date)}")
# Fetch the futures data for the selected expiry date
nft = fetch_futures(cursor, symbol, expiry_date)
# Convert the 'date_timestamp' column to datetime if it exists




Selected expiry date: 2023-05-25 of type <class 'datetime.date'>
Selected expiry date: 2023-05-25 of type <class 'datetime.date'>


In [6]:

short_window = 9
long_window = 26
nft['Short_EMA'] = nft['close'].ewm(span=short_window, adjust=False).mean()
nft['Long_EMA'] = nft['close'].ewm(span=long_window, adjust=False).mean()

nft['Signal'] = 0


In [7]:
for i in range(1, len(nft)):
    if nft['Short_EMA'].iloc[i] > nft['Long_EMA'].iloc[i] and nft['Short_EMA'].iloc[i-1] <= nft['Long_EMA'].iloc[i-1]:
        nft.at[nft.index[i], 'Signal'] = 1  # Buy signal
    elif nft['Short_EMA'].iloc[i] < nft['Long_EMA'].iloc[i] and nft['Short_EMA'].iloc[i-1] >= nft['Long_EMA'].iloc[i-1]:
        nft.at[nft.index[i], 'Signal'] = -1  # Sell signal

nft.head()


,symbol,date_timestamp,open,low,high,close,volume,expiry,expiry_type,id,Short_EMA,Long_EMA,Signal
0,BANKNIFTY,2023-04-28 09:15:00,4305995,4304000,4313995,4312890,62525,2023-05-25 14:30:00,I,46716386,4312890.000,4.312890e+06,0
1,BANKNIFTY,2023-04-28 09:16:00,4313000,4310355,4313270,4310500,23100,2023-05-25 14:30:00,I,46716387,4312412.000,4.312713e+06,-1
2,BANKNIFTY,2023-04-28 09:17:00,4310855,4310360,4312300,4310650,16600,2023-05-25 14:30:00,I,46716388,4312059.600,4.312560e+06,0
3,BANKNIFTY,2023-04-28 09:18:00,4310580,4309845,4311095,4309865,18875,2023-05-25 14:30:00,I,46716389,4311620.680,4.312361e+06,0
4,BANKNIFTY,2023-04-28 09:19:00,4310000,4304300,4310200,4304970,26400,2023-05-25 14:30:00,I,46716390,4310290.544,4.311813e+06,0


In [8]:

fig = make_subplots(rows=1, cols=1)

# Candlestick chart
candlestick = go.Candlestick(x=nft.index,
                             open=nft['open'],
                             high=nft['high'],
                             low=nft['low'],
                             close=nft['close'],
                             name='Candlesticks')
fig.add_trace(candlestick)

# Short EMA
short_ema = go.Scatter(x=nft.index, y=nft['Short_EMA'], mode='lines', name='Short EMA')
fig.add_trace(short_ema)

# Long EMA
long_ema = go.Scatter(x=nft.index, y=nft['Long_EMA'], mode='lines', name='Long EMA')
fig.add_trace(long_ema)

# Buy signals
buy_signals = go.Scatter(x=nft[nft['Signal'] == 1].index, 
                         y=nft['Short_EMA'][nft['Signal'] == 1], 
                         mode='markers', 
                         marker=dict(symbol='triangle-up', color='blue', size=8), 
                         name='Buy Signal')
fig.add_trace(buy_signals)

# Sell signals
sell_signals = go.Scatter(x=nft[nft['Signal'] == -1].index, 
                          y=nft['Short_EMA'][nft['Signal'] == -1], 
                          mode='markers', 
                          marker=dict(symbol='triangle-down', color='black', size=8), 
                          name='Sell Signal')
fig.add_trace(sell_signals)

fig.update_layout(
    title=f'{symbol} EMA Crossover Strategy',
    yaxis_title='Price',
    xaxis_title='Date',
    xaxis_rangeslider_visible=False,
    width=2000,
    height=1000
)

fig.show()
pio.write_html(fig,file='p.html',auto_open=True)


In [9]:

position_size = 1  
current_position = 0  # Track current position (1 for long, -1 for short, 0 for neutral)
entry_price = 0  # Track entry price
total_pnl = 0  # Total P&L
# Iterate through each row in the DataFrame
for index, row in nft.iterrows():
    if row['Signal'] == 1 and current_position == 0:  # Buy signal and no current position
        current_position = 1
        # print("entry")
        entry_price = row['close']
    elif row['Signal'] == -1 and current_position == 1:  # Sell signal and long position
        pnl = (row['close'] - entry_price) * position_size
        total_pnl += pnl
        # ###print("close")
        current_position = 0
        entry_price = 0
    elif row['Signal'] == -1 and current_position == 0:  # Sell signal but no current long position
        continue  # Can't sell if not long

# Print total P&L
print(f'Total P&L: {total_pnl}')

Total P&L: 51180


In [10]:

# Assuming fetch_options is defined to fetch data based on 'symbol' and 'expiry'
cursor1 = conn.cursor()
options = fetch_options(cursor1, symbol=symbol, expiry=str(expiry_date))

In [11]:
copt = options[options['opt_type']=='CE'].copy()
popt = options[options['opt_type']=='PE'].copy()

options

,symbol,date_timestamp,open,high,low,close,volume,opt_type,strike,expiry_type,id,expiry
0,BANKNIFTY,2023-05-02 09:15:00,14520,17480,14520,16385,150,PE,4190000,I,214388,2023-05-25
1,BANKNIFTY,2023-05-02 09:15:00,75030,75495,73900,75305,20000,CE,4300000,I,219317,2023-05-25
2,BANKNIFTY,2023-05-02 09:15:00,3085,3160,2865,2900,2125,CE,4600000,I,227633,2023-05-25
3,BANKNIFTY,2023-05-02 09:15:00,52305,54040,51400,53000,775,CE,4340000,I,222012,2023-05-25
4,BANKNIFTY,2023-05-02 09:15:00,82050,88405,82050,88405,400,CE,4280000,I,218230,2023-05-25
...,...,...,...,...,...,...,...,...,...,...,...,...
576973,BANKNIFTY,2023-05-25 15:29:00,5,5,5,5,1750,CE,4900000,I,14860719,2023-05-25
576974,BANKNIFTY,2023-05-25 15:29:00,10,15,10,10,69225,CE,4650000,I,14858713,2023-05-25
576975,BANKNIFTY,2023-05-25 15:29:00,31860,31860,31695,31800,39925,PE,4400000,I,14855946,2023-05-25
576976,BANKNIFTY,2023-05-25 15:29:00,5,15,5,5,2225,CE,4750000,I,14859590,2023-05-25


In [12]:
def binary_search(arr, target):
    low , high = 0, len(arr)
    while low < high:
        mid = low + (high -low)//2
        if arr['strike'].iloc[mid] < target:
            low = mid + 1
        else:
            high = mid
    return low


In [106]:
# # Define the structures
# put = {'pos': 0, 'signal_time': None, 'long_ema': None, 'short_ema': None, 'entry_time': None, 'strike': None, 'entry_price': None, 'exit_time': None, 'exit_price': None}
# call = {'pos': 0, 'signal_time': None, 'long_ema': None, 'short_ema': None, 'entry_time': None, 'strike': None, 'entry_price': None, 'exit_time': None, 'exit_price': None}

# # Initialize trades list
# trades = []
# pnl = []
# per_trade_data = []

# # Iterate over the rows of nft DataFrame
# for i in range(len(nft) - 1):
#     row = nft.iloc[i]
#     signal = row['Signal']
#     close = nft.iloc[i + 1]['close']
#     open = nft.iloc[i + 1]['open']
#     dt = nft.iloc[i + 1]['date_timestamp']

#     if signal == 1:
#         if call['pos'] == 0:
#             strikes = copt[copt['date_timestamp'] == dt][['date_timestamp', 'open', 'strike']]
#             if not strikes.empty:
#                 strikes.sort_values(by='strike', inplace=True)
#                 var = open
#                 strikes.reset_index(drop=True, inplace=True)
#                 index = binary_search(strikes, var)
#                 if index >= len(strikes):
#                     index -= 1
#                 # if index > 0:
#                 #     index -= 1
#                 buy_val = strikes['open'].iloc[index]
#                 call.update({'pos': 1, 'signal_time': row['date_timestamp'], 'long_ema': row['Long_EMA'],
#                              'short_ema': row['Short_EMA'], 'strike': strikes['strike'].iloc[index],
#                              'entry_time': dt, 'entry_price': buy_val})
#         if put['pos'] == 1:
#             strikes = popt[(popt['date_timestamp'] == dt) & (popt['strike'] == put['strike'])]
#             if not strikes.empty:
#                 var = strikes['open'].iloc[0]
#                 put.update({'pos': 0, 'exit_time': dt, 'exit_price': var})
#                 trades.append(put.copy())  # Ensure to copy the current state of put
#                 per_trade_data.append({
#                     'dts': dt,
#                     'buy': put['entry_price'],
#                     'sell': var,
#                     'pnl': var - put['entry_price']
#                 })
#             else:
#                 print('no closing on put opt')

#     elif signal == -1:
#         if put['pos'] == 0:
#             strikes = popt[popt['date_timestamp'] == dt][['date_timestamp', 'open', 'strike']]
#             if not strikes.empty:
#                 strikes.sort_values(by='strike', inplace=True)
#                 var = open
#                 strikes.reset_index(drop=True, inplace=True)
#                 index = binary_search(strikes, var)
#                 if index >= len(strikes):
#                     index -= 1
#                 # if index < len(strikes):
#                 #     index += 1
#                 sell_val = strikes['open'].iloc[index]
#                 put.update({'pos': 1, 'signal_time': row['date_timestamp'], 'long_ema': row['Long_EMA'],
#                             'short_ema': row['Short_EMA'], 'strike': strikes['strike'].iloc[index],
#                             'entry_time': dt, 'entry_price': sell_val})
#                 trades.append({
#                     'dts': dt,
#                     'strike': strikes['strike'].iloc[index],
#                     'pos': 'long',
#                     'type': 'PE',
#                     'price': sell_val
#                 })
#         if call['pos'] == 1:
#             strikes = copt[(copt['date_timestamp'] == dt) & (copt['strike'] == call['strike'])]
#             if not strikes.empty:
#                 var = strikes['open'].iloc[0]
#                 call.update({'pos': 0, 'exit_time': dt, 'exit_price': var})
#                 trades.append(call.copy())  # Ensure to copy the current state of call
#                 per_trade_data.append({
#                     'dts': dt,
#                     'buy': call['entry_price'],
#                     'sell': var,
#                     'pnl': var - call['entry_price']
#                 })
#             else:
#                 print('no closing on call opt')

# # Final closing for any open positions at the end of the dataset
# if call['pos'] == 1:
#     call_short = copt[(copt['date_timestamp'] == nft.iloc[-1]['date_timestamp']) & (copt['strike'] == call['strike'])]
#     if not call_short.empty:
#         print('short on call opt', end=' ')
#         var = call_short.iloc[0]['close']
#         call.update({'pos': 0, 'exit_time': nft.iloc[-1]['date_timestamp'], 'exit_price': var})
#         trades.append(call.copy())  # Ensure to copy the current state of call
#         per_trade_data.append({
#             'dts': nft.iloc[-1]['date_timestamp'],
#             'buy': call['entry_price'],
#             'sell': var,
#             'pnl': var - call['entry_price']
#         })
#     else:
#         print('no closing on call opt')

# if put['pos'] == 1:
#     put_short = popt[(popt['date_timestamp'] == nft.iloc[-1]['date_timestamp']) & (popt['strike'] == put['strike'])]
#     if not put_short.empty:
#         var = put_short.iloc[0]['close']
#         put.update({'pos': 0, 'exit_time': nft.iloc[-1]['date_timestamp'], 'exit_price': var})
#         trades.append(put.copy())  # Ensure to copy the current state of put
#         per_trade_data.append({
#             'dts': nft.iloc[-1]['date_timestamp'],
#             'buy': put['entry_price'],
#             'sell': var,
#             'pnl': var - put['entry_price']
#         })
#     else:
#         print('no closing on put opt')


no closing on put opt
no closing on put opt


In [138]:
# Define the structures
put = {'pos': 0, 'signal_time': None, 'long_ema': None, 'short_ema': None, 'entry_time': None, 'strike': None,
        'entry_price': None, 'exit_time': None, 'exit_price': None , 'pnl': 0 , 'pnl_sum':0}
call = {'pos': 0, 'signal_time': None, 'long_ema': None, 'short_ema': None, 'entry_time': None, 'strike': None,
         'entry_price': None, 'exit_time': None, 'exit_price': None , 'pnl': 0 , 'pnl_sum':0}

# Initialize trades list
trades = []
pnl = []
per_trade_data = []

# Iterate over the rows of nft DataFrame
for i in range(len(nft) - 1):
    row = nft.iloc[i]
    signal = row['Signal']
    close = nft.iloc[i + 1]['close']
    open = nft.iloc[i + 1]['open']
    dt = nft.iloc[i + 1]['date_timestamp']

    if signal == 1:
        if call['pos'] == 0:
            strikes = copt[copt['date_timestamp'] == dt][['date_timestamp', 'open', 'strike']]
            if not strikes.empty:
                strikes.sort_values(by='strike', inplace=True)
                var = open
                strikes.reset_index(drop=True, inplace=True)
                index = binary_search(strikes, var)
                if index >= len(strikes):
                    index -= 1
                # if index > 0:
                #     index -= 1
                buy_val = strikes['open'].iloc[index]
                call.update({'pos': 1, 'signal_time': row['date_timestamp'], 'long_ema': row['Long_EMA'],
                             'short_ema': row['Short_EMA'], 'strike': strikes['strike'].iloc[index],
                             'entry_time': dt, 'entry_price': buy_val})
        if put['pos'] == 1:
            strikes = popt[(popt['date_timestamp'] == dt) & (popt['strike'] == put['strike'])]
            if not strikes.empty:
                var = strikes['open'].iloc[0]
                put.update({'pos': 0, 'exit_time': dt, 'exit_price': var , 'pnl': var - put['entry_price']})
                trades.append(put.copy())  # Ensure to copy the current state of put
            else:
                print('no closing on put opt')

    elif signal == -1:
        if put['pos'] == 0:
            strikes = popt[popt['date_timestamp'] == dt][['date_timestamp', 'open', 'strike']]
            if not strikes.empty:
                strikes.sort_values(by='strike', inplace=True)
                var = open
                strikes.reset_index(drop=True, inplace=True)
                index = binary_search(strikes, var)
                if index >= len(strikes):
                    index -= 1
                # if index < len(strikes):
                #     index += 1
                sell_val = strikes['open'].iloc[index]
                put.update({'pos': 1, 'signal_time': row['date_timestamp'], 'long_ema': row['Long_EMA'],
                            'short_ema': row['Short_EMA'], 'strike': strikes['strike'].iloc[index],
                            'entry_time': dt, 'entry_price': sell_val})
        if call['pos'] == 1:
            strikes = copt[(copt['date_timestamp'] == dt) & (copt['strike'] == call['strike'])]
            if not strikes.empty:
                var = strikes['open'].iloc[0]
                call.update({'pos': 0, 'exit_time': dt, 'exit_price': var ,'pnl': var - call['entry_price']})
                trades.append(call.copy())  # Ensure to copy the current state of call
            else:
                print('no closing on call opt')

# Final closing for any open positions at the end of the dataset
if call['pos'] == 1:
    call_short = copt[(copt['date_timestamp'] == nft.iloc[-1]['date_timestamp']) & (copt['strike'] == call['strike'])]
    if not call_short.empty:
        print('short on call opt', end=' ')
        var = call_short.iloc[0]['close']
        call.update({'pos': 0, 'exit_time': nft.iloc[-1]['date_timestamp'], 'exit_price': var , 'pnl': var - call['entry_price']})
        trades.append(call.copy())  # Ensure to copy the current state of call
    else:
        print('no closing on call opt')

if put['pos'] == 1:
    put_short = popt[(popt['date_timestamp'] == nft.iloc[-1]['date_timestamp']) & (popt['strike'] == put['strike'])]
    if not put_short.empty:
        var = put_short.iloc[0]['close']
        put.update({'pos': 0, 'exit_time': nft.iloc[-1]['date_timestamp'], 'exit_price': var , 'pnl': var - put['entry_price']})
        trades.append(put.copy())  # Ensure to copy the current state of put
    else:
        print('no closing on put opt')


no closing on put opt
no closing on put opt


In [139]:
for i in range(1,len(trades)):
    trades[i]['pnl_sum'] = trades[i-1]['pnl_sum'] + trades[i]['pnl']

In [140]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=[i for i in range(len(trades))],
    y=[i['pnl'] for i in trades],
    mode='lines+markers',
    name='PnL'
))

fig.show()

In [141]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=[i for i in range(len(trades))],
    y=[i['pnl_sum'] for i in trades],
    mode='lines+markers',
    name='PnL'
))

fig.show()

In [142]:
def drawdown(arr):
    max_val = -10000000000
    max_index , i = 0 , 0
    drawdown_val = -10000000000
    while i < len(arr):
        if arr[i]['pnl_sum'] > max_val:
            max_val = arr[i]['pnl_sum']
            while i < len(arr) and arr[i]['pnl_sum']<=max_val:
                drawdown_val = max(drawdown_val, max_val - arr[i]['pnl_sum'])
                i += 1

    return drawdown_val


In [143]:
drawdownval = drawdown(trades)
print(drawdownval)

20395


In [144]:
print(trades[0])

{'pos': 0, 'signal_time': Timestamp('2023-05-02 09:45:00'), 'long_ema': np.float64(4342757.370624161), 'short_ema': np.float64(4342631.5558883045), 'entry_time': Timestamp('2023-05-02 09:46:00'), 'strike': np.int64(4350000), 'entry_price': np.int64(60980), 'exit_time': Timestamp('2023-05-02 10:06:00'), 'exit_price': np.int64(60400), 'pnl': np.int64(-580), 'pnl_sum': 0}


In [145]:
pnl_per_day = {}

for trade in trades:
    if trade['entry_time'] not in pnl_per_day:
        pnl_per_day[trade['entry_time']] = []
    pnl_per_day[trade['entry_time']].append(-trade['entry_price'])

    if trade['exit_time'] not in pnl_per_day:
        pnl_per_day[trade['exit_time']] = []
    pnl_per_day[trade['exit_time']].append(trade['exit_price'])


net_pnl_per_day = {}

for day in pnl_per_day:

    net_pnl_per_day[day] = sum(pnl_per_day[day])
    

In [146]:
entry_per_day = {}
exit_per_day = {}

for trade in trades:
    if trade['entry_time'] not in entry_per_day:
        entry_per_day[trade['entry_time']] = 0.0
    entry_per_day[trade['entry_time']] += trade['entry_price']

    if trade['exit_time'] not in exit_per_day:
        exit_per_day[trade['exit_time']] = 0.0
    exit_per_day[trade['exit_time']] += trade['exit_price']


In [151]:
percentage_pnl = {}

for day in entry_per_day:
    percentage_pnl[day] = (net_pnl_per_day[day] / entry_per_day[day]) * 100

In [152]:
arr = np.array(list(percentage_pnl.values()))

mean = np.mean(arr)

std_dev = np.std(arr)

risk_free_rate = 0.0

sharpe_ratio = (mean - risk_free_rate) / std_dev

print(sharpe_ratio)

0.04531391514713122
